<a href="https://colab.research.google.com/github/issildwg/Dissertation-Classifier/blob/main/Copy_of_Classifiern_with_val.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#takes data as a whole (full tweet) and puts it through the TF BERT model to create length vector

import tensorflow as tf
import tensorflow_hub as hub

!pip3 install tensorflow_text
import tensorflow_text as text


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 95.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd

df = pd.read_csv("Dataset.csv", error_bad_lines=False)
df.head()

<ipython-input-2-d4e7a6bddc51>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv("Dataset.csv", error_bad_lines=False)
<ipython-input-2-d4e7a6bddc51>:3: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Dataset.csv", error_bad_lines=False)


,Label,Tweet Uppercase,Tweet,Location
0,Extremist,_ the deep state always wins,_ the deep state always wins,NaN
1,Extremist,syria syrhash,syria syrhash,NaN
2,Extremist, algo falla, algo falla,NaN
3,Extremist,liberal response see pic,liberal response see pic,NaN
4,Extremist,alan what you say is bs trump is figh...,alan what you say is bs trump is figh...,NaN


In [ ]:
df = df.drop(['Tweet Uppercase'], axis=1)      #(index='cow', columns='small') doing index helps filter out - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html
df.head()

,Label,Tweet,Location
0,Extremist,_ the deep state always wins,NaN
1,Extremist,syria syrhash,NaN
2,Extremist, algo falla,NaN
3,Extremist,liberal response see pic,NaN
4,Extremist,alan what you say is bs trump is figh...,NaN


In [ ]:
df.groupby("Label").describe()  #shows most common tweet and frequency it was tweeted for each category

Tweet         \
               count unique   
Label                         
Extremist      93628  93592   
Non-Extremist    997    414   

                                                                       \
                                                             top freq   
Label                                                                   
Extremist                                             thank you     4   
Non-Extremist  elijahdaniel rip derrick scott theyre only rel...   97   

              Location                             
                 count unique            top freq  
Label                                              
Extremist            0      0            NaN  NaN  
Non-Extremist      488    349  United States   13

In [ ]:
# we are using a technique called downsampling (called it oversampling https://www.youtube.com/watch?v=JnlM4yLFNuo) to help deal with this imbalanced dataset (12754 vs 8432)
df["Label"].value_counts()

Non-Extremist    282165
Extremist         93628
Name: Label, dtype: int64

In [ ]:
df_extremist = df[df["Label"] == "Extremist"]   #creating data frames with only extremist tweets
df_extremist.shape        #shape shows the shape of the datatype (tuple) - (rows, columns)

(93628, 3)

In [ ]:
df_neutral = df[df["Label"] == "Non-Extremist"]   #creating data frames with only non-extremist/peaceful tweets
df_neutral.shape

(282165, 3)

extremist is larger

In [ ]:
df_extremist_downsampled = df_extremist.sample(df_neutral.shape[0]) #larger is getting downsampled here - sampling extremist dataframe - taking the same no of extremist tweets as non-extremist tweets - keep this dynamic as datasets may chenge
df_extremist_downsampled.shape

ValueError: ignored

In [ ]:
df_bal = pd.concat([df_extremist_downsampled, df_neutral]) #concat both new extremist and neutral together to form the balanced dataframe
df_bal.shape   #gives the new size of the dataset

neutral is larger

In [ ]:
df_neutral_downsampled = df_neutral.sample(df_extremist.shape[0]) #larger is getting downsampled here - sampling extremist dataframe - taking the same no of extremist tweets as non-extremist tweets - keep this dynamic as datasets may chenge
df_neutral_downsampled.shape

(93628, 3)

In [ ]:
df_bal = pd.concat([df_neutral_downsampled, df_extremist]) #concat both new extremist and neutral together to form the balanced dataframe
df_bal.shape   #gives the new size of the dataset

(187256, 3)

In [ ]:
df_bal["Label"].value_counts()  #now we have equal no of extremist and neutral tweets & discarded rest of data

Non-Extremist    93628
Extremist        93628
Name: Label, dtype: int64

In [ ]:
df_bal.sample(5)

,Label,Tweet,Location
184727,Non-Extremist,NaN,NaN
111203,Non-Extremist,NaN,NaN
266667,Non-Extremist,NaN,NaN
258560,Non-Extremist,NaN,NaN
109594,Non-Extremist,NaN,NaN


In [ ]:
df_bal["Extremist"] = df_bal["Label"].apply(lambda x: 1 if x=="Extremist" else 0)  #applying the apply function to the (above) tweet label column - lambda is a quick way of writing a simple function
#creating binary column to define extremism (ie 0 = non-extremist, 1 = extremist)
df_bal.sample(10)

,Label,Tweet,Location,Extremist
48074,Extremist,stay on message but enjoy yourself as well sir,NaN,1
276824,Non-Extremist,NaN,NaN,0
29303,Extremist,islamic terrorism neverforget september years...,NaN,1
97723,Non-Extremist,NaN,NaN,0
349538,Non-Extremist,NaN,NaN,0
26781,Extremist,if u have the time send me a good meme m,NaN,1
227615,Non-Extremist,NaN,NaN,0
181829,Non-Extremist,NaN,NaN,0
89256,Extremist,this outfit is the only tool that will strike ...,NaN,1
326828,Non-Extremist,NaN,NaN,0


In [ ]:
from sklearn.model_selection import train_test_split
x = df_bal[["Tweet", "Location"]]
y = df_bal["Extremist"]
x_train_val, x_test, y_train_val, y_test = train_test_split(x, y, stratify=y, test_size=0.2)
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, stratify=y_train_val, test_size=0.2)

In [ ]:
#these training models are available on tensorflow website
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")   #implementing the Bert pre-processind model https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4") #implementing the Bert Bidirectional Encoder model https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4

#Encoding tweets

In [ ]:
def get_data_embedding(tweets, locations):    #getting tweets and locations as input
  preprocessed_tweet = bert_preprocess(tweets)    #call preproccess on tweets and get preprocessed tweets as an output which is then given to bert_encoder
  preprocessed_loc = bert_preprocess(locations)

  embeddings_tweet = tf.convert_to_tensor(bert_encoder(preprocessed_tweet)["pooled_output"])
  embeddings_loc = tf.convert_to_tensor(bert_encoder(preprocessed_loc)["pooled_output"])
  return embeddings_tweet, embeddings_loc     #will return a dictionary - have to choose a pooled output


#def get_tweet_embedding(tweets):
#    embeddings = []
#    for tweet, location in tweets:
#        preprocessed_tweet = bert_preprocess(tweet)
#        embedding = bert_encoder(preprocessed_tweet)["pooled_output"]
#        embeddings.append((embedding, location))
#    return embeddings


**Bert Layers**

In [ ]:
#using functional approach
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")    #creating the input layer - shape: let it derive based on which encoding is being passed in  - shape: A shape tuple (integers), not including the batch size. For instance, shape=(32,) indicates that the expected input will be batches of 32-dimensional vectors.
loc_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="location")

preprocessed_tweet = bert_preprocess(text_input)  #pass the input layer just created to BERT preprocessing
preprocessed_loc = bert_preprocess(loc_input)  #pass the input layer just created to BERT preprocessing

outputs_tweet = bert_encoder(preprocessed_tweet)
outputs_loc = bert_encoder(preprocessed_loc)

outputs = tf.keras.layers.concatenate([outputs_tweet["pooled_output"], outputs_loc["pooled_output"]], axis=-1)




**Neural network layers**

In [ ]:
#creating layers
#concatenated = tf.keras.layers.concatenate([outputs_tweet["pooled_output"], outputs_loc["pooled_output"]], axis=-1)
#layer = tf.keras.layers.Dropout(0.1, name="dropout")(concatenated)  #since it is a functional style neural network - call as if this is a function - requires an input (which is the output of the previous layer) - the pooled_output has the word encoding of 768 size vector


layer = tf.keras.layers.Dropout(0.1, name="dropout")(outputs)  #since it is a functional style neural network - call as if this is a function - requires an input (which is the output of the previous layer) - the pooled_output has the word encoding of 768 size vector
layer = tf.keras.layers.Dense(1, activation="sigmoid", name="output")(layer)  # has only one neuron - is it extremist or not


**Constructing final model**

In [ ]:
model = tf.keras.Model(inputs=[text_input, loc_input], outputs=[layer])  #this is where you would be able to put multiple input and output layers

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 location (InputLayer)          [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]',                   
                                (None, 128),                      'location[0][0]']               
                                 'input_word_ids':                                                
                                (None, 128),                                                  

In [ ]:
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name="accuracy"),
    tf.keras.metrics.Precision(name="precision"),
    tf.keras.metrics.Recall(name="recall")
]

model.compile(optimizer="adam",             #implementing the Adam algorithm - https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam
              loss="binary_crossentropy",   #since output is binary - this is the bit that influences the training (since this is driven by the loss function)
              metrics=METRICS)

In [ ]:
x_train["Tweet"] = x_train["Tweet"].astype(str)
x_train["Location"] = x_train["Location"].astype(str)

# Train the model
model.fit([tf.convert_to_tensor(x_train["Tweet"]), tf.convert_to_tensor(x_train["Location"])], y_train,
                    epochs=8, batch_size=32#, validation_data=([tf.convert_to_tensor(x_val["Tweet"]), tf.convert_to_tensor(x_val["Location"])], y_val)
                    )


Epoch 1/8
3746/3746 [==============================] - 2268s 601ms/step - loss: 0.0410 - accuracy: 0.9901 - precision: 0.9971 - recall: 0.9831
Epoch 2/8
3746/3746 [==============================] - 2253s 602ms/step - loss: 0.0155 - accuracy: 0.9970 - precision: 0.9981 - recall: 0.9960
Epoch 3/8
3746/3746 [==============================] - 2254s 602ms/step - loss: 0.0118 - accuracy: 0.9979 - precision: 0.9982 - recall: 0.9976
Epoch 4/8
 555/3746 [===>..........................] - ETA: 31:59 - loss: 0.0085 - accuracy: 0.9984 - precision: 0.9987 - recall: 0.9982

In [ ]:
x_val["Tweet"] = x_val["Tweet"].astype(str)
x_val["Location"] = x_val["Location"].astype(str)

model.fit([tf.convert_to_tensor(x_val["Tweet"]), tf.convert_to_tensor(x_val["Location"])], y_val, epochs=1)

In [ ]:
x_test["Tweet"] = x_test["Tweet"].astype(str)
x_test["Location"] = x_test["Location"].astype(str)

model.fit([tf.convert_to_tensor(x_test["Tweet"]), tf.convert_to_tensor(x_test["Location"])], y_test, epochs=1)
#model.fit([tf.convert_to_tensor(x_train["Tweet"].values), tf.convert_to_tensor(x_train["Location"].values)], y_train, epochs=15)
#model.fit([x_train["Tweet"], x_train["Location"]], y_train, epochs=15)    #one epoch = one entire passing of training data through the algorithm  - doing this 15 times avoid overfitting but is still specific
#loss - a number indicating how bad the model's prediction was on a single example. If the model's prediction is perfect, the loss is zero; otherwise, the loss is greater.
#accuracy - this function computes subset accuracy: the set of labels predicted for a sample must exactly match the corresponding set of labels in y_true.
#precision - The precision is the ratio true positives / (true pos + false pos). The precision is intuitively the ability of the classifier not to label as positive a sample that is negative.
#recall - The recall is the ratio true positives / (true pos + false negatives). The recall is intuitively the ability of the classifier to find all the positive samples.

In [ ]:
x_test = pd.concat([x_test["Tweet"], x_test["Location"]])

# Convert the DataFrame to a numpy array
x_test = np.array(x_test)

# Make predictions
y_predicted = model.predict(x_test)

In [ ]:
import numpy as np
y_predicted = model.predict(x_test["Tweet"], x_test["Location"])


y_predicted =np.where(y_predicted > 0.5, 1, 0)  #y_predicted is a bunch of sigmoid values, (ie 0.3, 0.6, 0.9 etc) this rounds them to 0 if they are below 0.5, and 1 if theyre above
y_predicted

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

#cm = confusion_matrix(y_test, y_predicted)
#cm

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sn
sn.heatmap(cm, annot=True, fmt="d")
plt.xlabel("predicted")
plt.ylabel("true")
#this is the same matrix as above in cell 34, but with prettier visuals (from seaborn)
#ie TL - number of times it was neutal, and was predicted to be neutral, BL was extremist, predicted neutral, TR was neutral, predicted extremist, BR was extremist and predicted extremist

**mention this** - f1 score is also accuracy score


In [ ]:
print(classification_report(y_test))
#mid accuracy, mid precision, okay recall - the f1 score shoud be high

In [ ]:
#more than 0.5 means this is extremist, less means it is neutral
e = [
    "war is good",
    "fear is bad",
    "ISIS is the best",
    "thank god for kim jong un",
    "i hate kids"
]
model.predict(e)

In [ ]:
#@title ignore this bit
def create_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
    loc_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="location")

    embeddings_tweet, embeddings_loc = get_tweet_embedding([text_input, loc_input])

    # concatenate the tweet and location embeddings
    concatenated = tf.keras.layers.concatenate([embeddings_tweet, embeddings_loc], axis=-1)

    # apply dropout and output layer
    layer = tf.keras.layers.Dropout(0.1, name="dropout")(concatenated)
    layer = tf.keras.layers.Dense(1, activation="sigmoid", name="output")(layer)

    # compile the model
    METRICS = [
        tf.keras.metrics.BinaryAccuracy(name="accuracy"),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall")
    ]

    model = tf.keras.Model(inputs=[text_input, loc_input], outputs=[layer])
    model.compile(optimizer="adam",
                  loss="binary_crossentropy",
                  metrics=METRICS)

    return model

model = create_model()
model.fit(x=[x_train_tweet, x_train_loc], y=y_train, epochs=15)
